In [1]:
import pandas as pd
import optuna
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import MinMaxScaler
from xgboost import XGBClassifier
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, roc_auc_score, confusion_matrix
import numpy as np

# Load and preprocess dataset
df = pd.read_csv('/kaggle/input/transaction-dataset/transaction_dataset.csv')
df.drop(columns=['Unnamed: 0', 'Index'], inplace=True, errors='ignore')
df.fillna(0, inplace=True)

# Convert object columns to categorical
for col in df.select_dtypes(include='object').columns:
    df[col] = pd.Categorical(df[col])
    df[col] = df[col].astype('category')

# Scale numeric columns
numeric_cols = df.select_dtypes(include=['float64', 'int64']).columns
scaler = MinMaxScaler()
df[numeric_cols] = scaler.fit_transform(df[numeric_cols])

# Prepare features and target
X = df.drop(columns=['FLAG'])
y = df['FLAG']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

# Optuna objective
def objective(trial):
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 100, 300),
        "max_depth": trial.suggest_int("max_depth", 3, 12),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3),
        "subsample": trial.suggest_float("subsample", 0.6, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.6, 1.0),
        "gamma": trial.suggest_float("gamma", 0, 5),
        "reg_alpha": trial.suggest_float("reg_alpha", 0, 1),
        "reg_lambda": trial.suggest_float("reg_lambda", 0, 1)
    }

    model = XGBClassifier(
        use_label_encoder=False,
        eval_metric='logloss',
        enable_categorical=True,
        **params
    )

    score = cross_val_score(model, X_train, y_train, cv=5, scoring="roc_auc", error_score='raise').mean()
    return score

# Run Optuna study
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=100)

print("Best AUC:", study.best_value)
print("Best Parameters:", study.best_params)

# Train final model
best_model = XGBClassifier(
    use_label_encoder=False,
    eval_metric='logloss',
    enable_categorical=True,
    **study.best_params
)
best_model.fit(X_train, y_train)

# Predict
y_pred = best_model.predict(X_test)
y_proba = best_model.predict_proba(X_test)[:, 1]

# Performance Metrics
metrics = {
    "Model": ["XGBoost (Optuna)"],
    "Accuracy": [accuracy_score(y_test, y_pred)],
    "Precision": [precision_score(y_test, y_pred)],
    "Recall": [recall_score(y_test, y_pred)],
    "F1 Score": [f1_score(y_test, y_pred)],
    "ROC AUC": [roc_auc_score(y_test, y_proba)]
}
results_df = pd.DataFrame(metrics)
print("\nModel Performance:")
print(results_df)

# Confusion Matrix Breakdown
TP = np.logical_and(y_test == 1, y_pred == 1).sum()
TN = np.logical_and(y_test == 0, y_pred == 0).sum()
FP = np.logical_and(y_test == 0, y_pred == 1).sum()
FN = np.logical_and(y_test == 1, y_pred == 0).sum()

total = len(y_test)
print("\nPrediction Breakdown:")
print(f"✅ True Positives (Fraud correctly detected): {TP} ({TP/total:.2%})")
print(f"✅ True Negatives (Non-fraud correctly detected): {TN} ({TN/total:.2%})")
print(f"❌ False Positives (Non-fraud wrongly flagged as fraud): {FP} ({FP/total:.2%})")
print(f"❌ False Negatives (Fraud missed): {FN} ({FN/total:.2%})")

# Per-Class Accuracy
class_0_acc = TN / (TN + FP) if (TN + FP) > 0 else 0
class_1_acc = TP / (TP + FN) if (TP + FN) > 0 else 0

print("\nPer-Class Accuracy:")
print(f"Class 0 (Non-Fraud) Accuracy: {class_0_acc:.2%}")
print(f"Class 1 (Fraud) Accuracy: {class_1_acc:.2%}")


[I 2025-05-15 17:21:20,099] A new study created in memory with name: no-name-68b23dab-5450-470b-92f3-95f7f38bc97f
[I 2025-05-15 17:21:23,276] Trial 0 finished with value: 0.9992749962407078 and parameters: {'n_estimators': 127, 'max_depth': 11, 'learning_rate': 0.14561827038756842, 'subsample': 0.622980711267771, 'colsample_bytree': 0.6312059576432167, 'gamma': 2.833566476495348, 'reg_alpha': 0.9582691752078437, 'reg_lambda': 0.5162778615808405}. Best is trial 0 with value: 0.9992749962407078.
[I 2025-05-15 17:21:27,651] Trial 1 finished with value: 0.9995300943819105 and parameters: {'n_estimators': 227, 'max_depth': 11, 'learning_rate': 0.1070596829884972, 'subsample': 0.7181235354203521, 'colsample_bytree': 0.6885180681468925, 'gamma': 0.39440673020490313, 'reg_alpha': 0.5763288666662171, 'reg_lambda': 0.08815735720307627}. Best is trial 1 with value: 0.9995300943819105.
[I 2025-05-15 17:21:31,572] Trial 2 finished with value: 0.9994758039895577 and parameters: {'n_estimators': 127,

Best AUC: 0.9995792924819329
Best Parameters: {'n_estimators': 223, 'max_depth': 8, 'learning_rate': 0.15727982732682544, 'subsample': 0.8973796219445721, 'colsample_bytree': 0.9438642157201211, 'gamma': 0.4788944446772905, 'reg_alpha': 0.33559590847988213, 'reg_lambda': 0.2626952924609184}

Model Performance:
              Model  Accuracy  Precision    Recall  F1 Score   ROC AUC
0  XGBoost (Optuna)  0.995429    0.99536  0.983945  0.989619  0.999593

Prediction Breakdown:
✅ True Positives (Fraud correctly detected): 429 (21.79%)
✅ True Negatives (Non-fraud correctly detected): 1531 (77.76%)
❌ False Positives (Non-fraud wrongly flagged as fraud): 2 (0.10%)
❌ False Negatives (Fraud missed): 7 (0.36%)

Per-Class Accuracy:
Class 0 (Non-Fraud) Accuracy: 99.87%
Class 1 (Fraud) Accuracy: 98.39%
